# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRa
* Model: bert-base-cased
* Evaluation approach: accuracy
* Fine-tuning dataset:  imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import Dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# foundation model (DistilBERT) 
model_name = "bert-base-cased"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

dataset = load_dataset("imdb")

# Tokenize the dataset
def tokenize_batch(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

# Split the tokenized dataset into training and evaluation sets
#split_ratio = 0.8
#split_index = int(len(tokenized_dataset["input_ids"]) * split_ratio)

#train_dataset = {
 #   "input_ids": tokenized_dataset["input_ids"][:split_index],
  #  "attention_mask": tokenized_dataset["attention_mask"][:split_index],
   # "label": tokenized_dataset["label"][:split_index],
#}
#eval_dataset = {
 #   "input_ids": tokenized_dataset["input_ids"][split_index:],
  #  "attention_mask": tokenized_dataset["attention_mask"][split_index:],
   # "label": tokenized_dataset["label"][split_index:],
#}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 25000/25000 [00:27<00:00, 895.60 examples/s]


In [3]:
# Convert datasets to PyTorch Dataset objects
#class CustomDataset(Dataset):
 #   def __init__(self, input_ids, attention_mask, label):
  #      self.input_ids = input_ids
   #     self.attention_mask = attention_mask
    #    self.label = label

    #def __len__(self):
     #   return len(self.input_ids)

    #def __getitem__(self, idx):
     #   return {
      #      "input_ids": self.input_ids[idx],
       #     "attention_mask": self.attention_mask[idx],
        #    "label": self.label[idx],
       # }

In [4]:
#import numpy as np
#import evaluate
#!pip install evaluate
#metric = evaluate.load("accuracy")

In [5]:
#train_dataset = CustomDataset(**train_dataset)
#eval_dataset = CustomDataset(**eval_dataset)
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))
# Define the evaluation function for the Trainer
#def compute_metrics(p):
#    return {"accuracy": (p.predictions.argmax(axis=1) == p.label_ids).mean()}
import numpy as np
!pip install scikit-learn
from sklearn.metrics import accuracy_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Trainer for foundation model evaluation
training_args_foundation = TrainingArguments(
    output_dir="./foundation_output",
    per_device_eval_batch_size=8,
   
)
trainer_foundation = Trainer(
    model=model,
    args=training_args_foundation,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
foundation_results = trainer_foundation.evaluate()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [6]:
foundation_results

{'eval_loss': 0.7155689597129822,
 'eval_accuracy': 0.48,
 'eval_runtime': 33.5505,
 'eval_samples_per_second': 29.806,
 'eval_steps_per_second': 3.726}

In [7]:
foundation_eval_results = trainer_foundation.evaluate()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
# EFT model 
#peft_model = AutoModelForSequenceClassification.from_pretrained(model_name)  

In [9]:
# training arguments for PEFT
#training_args_peft = TrainingArguments(
 #   output_dir="./peft_output",
 #   evaluation_strategy="epoch",
 #   learning_rate=2e-5,
  #  per_device_train_batch_size=8,
  #  num_train_epochs=5,
#)

In [10]:
# Trainer for PEFT
#trainer_peft = Trainer(
 #   model=peft_model,
#    args=training_args_peft,
#    compute_metrics=compute_metrics,
 #   train_dataset=train_dataset,
 #   eval_dataset=eval_dataset,
#)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
#trainer_peft.train()

# Evaluate the PEFT model 
#peft_results = trainer_peft.evaluate()

# Compare results with the foundation model's performance
#print("Foundation Model Results:", foundation_results['eval_accuracy'])
#print("PEFT Model Results:", peft_results['eval_accuracy'])


In [12]:
#from datasets import load_dataset

#dataset = load_dataset("imdb")

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:56<00:00, 892.20 examples/s]


In [14]:
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [15]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [16]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [18]:
def compute_metrics(p):
    return {"accuracy": (p.predictions.argmax(axis=1) == p.label_ids).mean()}

In [19]:
#def compute_metrics(eval_pred):
 #   logits, labels = eval_pred
 #   predictions = np.argmax(logits, axis=-1)
 #   return metric.compute(predictions=predictions, references=labels)

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=3,)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset ,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.688352,0.512000
2,No log,0.684216,0.594000
3,No log,0.683215,0.565000


TrainOutput(global_step=375, training_loss=0.701417236328125, metrics={'train_runtime': 321.2695, 'train_samples_per_second': 9.338, 'train_steps_per_second': 1.167, 'total_flos': 789687078912000.0, 'train_loss': 0.701417236328125, 'epoch': 3.0})

In [23]:
peft_eval_results = trainer.evaluate()

In [24]:
print("Comparison of Pre-trained and PEFT Models:")
print("Pre-trained Model:", foundation_eval_results)
print("PEFT Model:", peft_eval_results)

Comparison of Pre-trained and PEFT Models:
Pre-trained Model: {'eval_loss': 0.7155689597129822, 'eval_accuracy': 0.48, 'eval_runtime': 33.7997, 'eval_samples_per_second': 29.586, 'eval_steps_per_second': 3.698}
PEFT Model: {'eval_loss': 0.6832150220870972, 'eval_accuracy': 0.565, 'eval_runtime': 33.7877, 'eval_samples_per_second': 29.597, 'eval_steps_per_second': 3.7, 'epoch': 3.0}
